# Not a Lawyer | Remote

Using: 
* Pinecone (https://www.pinecone.io/) as vectorstore
* OpenAI Embeddings
* OpenAI API as LLM

In [1]:
! pip freeze

aiofiles==23.2.1
aiohttp==3.9.1
aiosignal==1.3.1
altair==5.2.0
annotated-types==0.6.0
anyio==4.2.0
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1649077682618/work
asgiref==3.7.2
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
async-timeout==4.0.3
attrs==23.2.0
backoff==2.2.1
bcrypt==4.1.2
build==1.0.3
cachetools==5.3.2
certifi==2023.11.17
charset-normalizer==3.3.2
chroma-hnswlib==0.7.3
chromadb==0.4.22
click==8.1.7
colorama==0.4.6
coloredlogs==15.0.1
comm @ file:///home/conda/feedstock_root/build_artifacts/comm_1704278392174/work
contourpy==1.2.0
cycler==0.12.1
dataclasses-json==0.6.3
debugpy @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_563_nwtkoc/croot/debugpy_1690905063850/work
decorator @ file:///home/conda/feedstock_root/build_artifacts/decorator_1641555617451/work
Deprecated==1.2.14
distro==1.9.0
exceptiongroup @ file:///home/conda/feedstock_root/build_artifacts/exceptiongroup_1704921103267/

Before you start, you will need to set the following environmental variables on your computer:

**Pinecone**
* PINECONE_API_KEY
* PINECONE_ENVIRONMENT
* PINECONE_INDEX

**OpenAI**
* OPENAI_API_KEY


To add to your env variables on a mac, run this in your terminal:

```
export PINECONE_API_KEY=your_api_key PINECONE_ENVIRONMENT=env PINECONE_INDEX=index OPENAI_API_KEY=your_api_key
```

----

### Step 1: Import Libraries

In [ ]:
! pip install langchain langchain_openai gradio lxml pinecone-client

In [ ]:
# Import Libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter
from langchain.vectorstores import Pinecone
from langchain.prompts import PromptTemplate 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import gradio as gr

### Step 2: Define llm and embedding models
I'll use two variants: OpenAI and local model via LM Studio.

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, openai_api_key="")

# To use a local model through LM Studio, set llm like the commented line below
# llm = ChatOpenAI(base_url="http://localhost:1234/v1", temperature=0)

In [ ]:
embedding = OpenAIEmbeddings(openai_api_key="")


### Step 3: Process Data & Set up Vector Database with Pinecone

#### Step 3.1: Define the data (URLs in this case) for the vector database

Note: if you want to include different information in your RAG system, this is where you can add it.

In [ ]:
# German Residence Laws

aufentv =  "https://www.gesetze-im-internet.de/aufenthv/BJNR294510004.html"
aufenthg = "https://www.gesetze-im-internet.de/aufenthg_2004/BJNR195010004.html"

# Put in an array so that we can loop over them
urls = [aufentv, aufenthg]

#### Step 3.2: Split by HTML Headers
We split by HTML headers, as the hierarchy of headers is a good indicator of the structure of the document. We use the following headers: h1, h2, h3

In [ ]:
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

docs = []

for url in urls:
    html_header_splits = html_splitter.split_text_from_url(url)
    docs += html_header_splits


chunk_size = 1000
chunk_overlap = 200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(html_header_splits)


In [ ]:
# Sanity Check: are there any splits (this should be non-zero)
len(splits)

#### Step 3.3: Create a vector database with Pinecone
You need to create a pinecone account, and create an index: https://app.pinecone.io

In [ ]:
# First time here? Upsert the documents to the vector store (you only need to do this once)

# vectorstore = Pinecone.from_documents(
#     documents=splits, embedding=OpenAIEmbeddings(), index_name="lawyer"
# )


In [ ]:
# Do you already have your documents in the vector store? Then just load it:
vectorstore = Pinecone.from_existing_index("lawyer", OpenAIEmbeddings())

In [ ]:
# Connect your retriever to the vector store
retriever = vectorstore.as_retriever()

-----

### Step 4: Set up the Prompt

#### Step 4.1: Define the prompt template

In [ ]:
template = """
        ###INSTRUCTIONS: 
        You are polite and professional question-answering AI assistant. You must provide a helpful response to the user. 
        
        In your response, PLEASE ALWAYS:
          (0) Be a detail-oriented reader: read the question and context and understand both before answering
          (1) Start your answer with a friendly tone, and reiterate the question so the user is sure you understood it
          (2) If the context enables you to answer the question, write a detailed, helpful, and easily understandable answer with sources referenced inline. IF NOT: you can't find the answer, respond with an explanation, starting with: "I couldn't find the information in the laws I have access to". 
          (3) Below the answer, please list out all the referenced sources (i.e. legal paragraphs backing up your claims)
          (4) Now you have your answer, that's amazing - review your answer to make sure it answers the question, is helpful and professional and formatted to be easily readable.
        
        Think step by step. 
        ###
        
      Answer the following question using the context provided.
        ### Question: {question} ###

        ### Context: {context} ###

        

        ### Helpful Answer with Sources:

        """

    # create prompt template
prompt = PromptTemplate.from_template(template)

#### Step 4.2: Create the Chain

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Check that it's working by running the following cell:

In [ ]:
ans = chain.invoke("How do I get a blue card in Germany?")

print(ans)

-----

### Step 5: Set up Simple UI using Gradio

#### Step 5.1: Create a function to use in Gradio

In [ ]:
# Creating a function that takes a question and returns an answer is a good idea since 
# I'll make a gradio UI in the next cell, and this simplifies it
def get_answer(question):
    answer = chain.invoke(question)
    return answer

#### Step 5.2: Create and run the Gradio interface

In [ ]:

iface = gr.Interface(fn=get_answer, inputs=gr.Textbox(
    value="Enter your question"),
    live=True, 
    outputs="markdown",  
    title="LLM Augmented Q&A over German Residence Laws",
    description="Ask a question about German Residence Laws and get an answer from a friendly AI assistant. This assistant looks up relevant German Residence laws and answers your question.",
    examples=[["How do I get a blue card in Germany?"], 
            ["How long can I stay in Germany with a tourist visa?"],
            ["How do I get a work visa in Germany?"],["I want to work in Germany, what visa do I need?"],
            ["I am a student in Germany, can I work?"]],
    theme=gr.themes.Soft(),
    allow_flagging="never",)

iface.launch()